<a href="https://colab.research.google.com/github/mansukim1125/Deep-Learning-from-Scratch/blob/main/backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

가중치 매개변수에 대한 손실 함수의 기울기를 구하는 방법은 수치 미분을 사용하였다. 하지만 이는 구현하기는 쉽지만 속도가 느리다는 단점이 있다. 이를 개선하기 위해 ***오차역전파법***이 등장하게 된다. 이는 가중치 매개변수의 기울기를 효과적으로 계산하는 방법이다.

# 계산 그래프
계산 그래프는 계산 과정을 그래프로 나타낸 것이다. 다음의 예를 보자:

문제 1: 현빈 군은 슈퍼에서 1개에 100원인 사과를 2개 샀습니다. 이때 지불 금액을 구하세요. 단, 소비세가 10% 부과됩니다.

# 역전파
## 곱셈 노드의 역전파
$z=xy$라는 식을 생각해 보자. 이 식의 미분은 다음과 같다.

$\frac{\partial z}{\partial x}=y, \frac{\partial z}{\partial y}=x$

이처럼 곱셈 노드 역전파는 상류의 값($z$)에 순전파 때의 입력 신호를 서로 바꾸어 곱해 하류로 보낸다.

## 덧셈 노드의 역전파
$z=x+y$라는 식을 생각해 보자. 이 식의 미분은 다음과 같다.

$\frac{\partial z}{\partial x}=1, \frac{\partial z}{\partial y}=1$

이처럼 덧셈 노드 역전파는 상류의 값에 1을 곱하고 하류로 보낸다. 따라서 상류의 값이 그대로 하류로 흐른다는 것을 알 수 있다.

# 단순한 계층 구현하기
곱셈 계층(`MulLayer`)와 덧셈 계층(`AddLayer`)를 구현해 보자.

## 곱셈 계층
모든 계층은 `forward()`와 `backward()`라는 공통의 메서드를 갖도록 구현할 것이다. `forward()`는 순전파, `backward()`는 역전파를 의미한다.

In [ ]:
class MulLayer:
  def __init__(self):
    self.x = None
    self.y = None

  def forward(self, x, y):
    """순전파는 x, y를 곱해 반환합니다"""
    self.x = x
    self.y = y
    out = x * y

    return out

  def backward(self, dout):
    """역전파는 상류의 값에 x, y를 바꾸어 곱한 후 반환합니다"""
    dx = dout * self.y
    dy = dout * self.x

    return dx, dy

그렇다면 161쪽의 그림 5-16 '사과 2개 구입' 예를 순전파와 역전파를 이용해 구현해 보자:

In [ ]:
apple = 100
apple_quantity = 2
tax = 1.1

apple_apple_quantity_layer = MulLayer()
apple_apple_quantity_tax_layer = MulLayer()

apple_price = apple_apple_quantity_layer.forward(apple, apple_quantity)
total_price = apple_apple_quantity_tax_layer.forward(apple_price, tax)

print(total_price)

220.00000000000003


또한 각 변수에 대한 미분은 `backward()`로 구할 수 있다.

In [ ]:
dtotal_price = 1

dapple_price, dtax = apple_apple_quantity_tax_layer.backward(dtotal_price)
dapple, dapple_num = apple_apple_quantity_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


## 덧셈 계층
덧셈 계틍은 상류에서 내려온 미분을 그대로 하류로 보낸다.

In [ ]:
class AddLayer:
  def __init__(self):
    pass

  def forward(self, x, y):
    """순전파는 입력된 x, y를 더해 반환합니다"""
    out = x + y
    return out

  def backward(self, dout):
    """역전파는 상류의 미분 값을 그대로 하류로 반환합니다"""
    dx = dout * 1
    dy = dout * 1
    return dx, dy

(163쪽 그림 5-17 구현하기)

# 활성화 함수 계층 구현하기
계산 그래프를 신경망에 적용해 보자. 먼저 활성화 함수인 ReLU와 Sigmoid 계층을 구현해 보자.

## ReLU 계층
수식은 다음과 같다:

$
y=\left\{\begin{matrix}
x & (x>0) \\
0 & (x\leq 0) \\
\end{matrix}\right.
$

이다. 위 식의 $\frac{\partial y}{\partial x}$은 아래와 같다:

$
\frac{\partial y}{\partial x}=\left\{\begin{matrix}
1 & (x>0) \\
0 & (x\leq 0) \\
\end{matrix}\right.
$

이와 같이 순전파 때의 입력($x$)가 0보다 크면 역전파는 상류의 값을 그대로 하류로 보낸다. 또 0 이하이면 0을 곱해 하류로 신호를 보내지 않는다. 이 ReLU 계층을 코드로 구현하면 다음과 같이 표현할 수 있다:

In [ ]:
class ReLU:
  def __init__(self):
    self.mask = None

  def forward(self, x):
    self.mask = (x <= 0) # 입력이 0이하인 원소는 True, 0보다 큰 원소는 False로 이루어진 np.array를 만듬
    out = x.copy()
    out[self.mask] = 0 # 입력이 0이하인 원소의 위치의 값은 0이 되어 신호를 보내지 않음

    return out

  def backward(self, dout):
    dout[self.mask] = 0 # 상류에서 넘어온 값(dout) 중 0이하인 원소의 위치의 값을 0으로 세트
    dx = dout

    return dx # 한 후 하류로 반환